In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/DL/Datasets/train_images.zip

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def read_pts(filename):
    return np.loadtxt(filename, comments=("version:", "n_points:", "{", "}"))

In [ ]:
from torch.utils.data import Dataset, DataLoader
class FacePointsDataset(Dataset):
  def __init__(self,root_dir, transform=None):
    self.root_dir = root_dir
    self.transform = transform
    self.images = os.listdir(self.root_dir+'/images')
    self.annot = os.listdir(self.root_dir+'/annot')

  def __len__(self):
    return len(self.images)
    
  def __getitem__(self, idx):
    image_name = self.root_dir+'/images/'+self.images[idx]
    if image_name.startswith("."):
      pass
    image = Image.open(image_name)
    image = image.convert('RGB')
    key_pts = read_pts(self.root_dir+"/annot/"+self.images[idx].split(".")[0]+".pts")
    key_pts = key_pts.astype('float')
    for i in range(len(key_pts)):
      key_pts[i][0] = 224*(key_pts[i][0])/image.size[0]
      key_pts[i][1] = 224*(key_pts[i][1])/image.size[1]
    if self.transform:
      image = self.transform(image)
    return image, key_pts

In [ ]:
from torchvision import transforms, utils
#defining transformations using transforms.Compose([])
data_transform =   transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = [0.5, 0.5, 0.5],std = [0.5, 0.5, 0.5])])
#creating dataset, by instantiating FacialKeyPointsDataset class
dataset = FacePointsDataset('/content/01_Indoor',transform=data_transform)

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import torch

In [ ]:
import torch.optim as optim
from tqdm import tqdm
num_epochs = 10
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CustomFacePointsModel(nn.Module):
    def __init__(self):
        super(CustomFacePointsModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.fc1 = nn.Linear(512*7*7, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 68*2)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc3(x)
        return x


In [ ]:
import torch.optim as optim
from tqdm import tqdm

# define your model
model = CustomFacePointsModel()

# define your loss function
criterion = nn.MSELoss()

# define your optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# set your model to training mode
model.train()

# train your model
num_epochs = 100
for epoch in range(num_epochs):
    loop = tqdm(enumerate(train_dataloader),total=len(train_dataloader))
    for i, data in loop:
        # get the inputs and labels
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        labels = labels.reshape(-1,2*68)
        # forward + backward + optimize
        outputs = model(inputs)
        outputs = outputs.type(torch.float64)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())

print('Finished Training')


100%|██████████| 6/6 [00:53<00:00,  8.85s/it, loss=2.38e+3]

Finished Training


In [ ]:
torch.save(model,'/content/model.pth')

In [ ]:
!cp /content/model.pth /content/drive/MyDrive/DL

cp: cannot stat '/content/model.pth': No such file or directory


In [ ]:
img=Image.open('/content/01_Indoor/images/indoor_001.png')
img_t=transforms.Resize((224,224))(img)

In [ ]:
img_t=np.asarray(img_t)
img_t.shape

(224, 224, 3)

In [ ]:
pts=read_pts('/content/01_Indoor/annot/indoor_001.pts')

In [ ]:
for i in range(len(pts)):
  pts[i][0] = 224*(pts[i][0])/img.size[0]
  pts[i][1] = 224*(pts[i][1])/img.size[1]

In [ ]:
pts

array([[ 99.5059761 ,  27.07038513],
       [100.27770518,  35.5020664 ],
       [100.61192032,  44.80416467],
       [102.75263745,  52.64922178],
       [105.19209562,  60.1370093 ],
       [108.89590438,  67.48022311],
       [112.896     ,  73.30957769],
       [117.05606375,  78.49370518],
       [123.44876494,  80.74560425],
       [129.11949004,  79.20616202],
       [133.7219761 ,  74.06159894],
       [137.42243825,  65.63854449],
       [139.98639044,  58.90010093],
       [141.75697211,  53.26737849],
       [143.24911554,  46.51673838],
       [144.4179761 ,  37.11111819],
       [144.24305976,  27.64867995],
       [104.92101992,  35.06358566],
       [108.47891633,  32.54841965],
       [112.244749  ,  34.04740505],
       [115.95212749,  35.72398406],
       [119.06804781,  37.96071182],
       [127.6034741 ,  36.76515272],
       [130.6745498 ,  34.85088977],
       [134.38750598,  32.44579017],
       [138.34387251,  30.92300664],
       [141.44216733,  33.09012483],
 

In [ ]:
import cv2
for i in pts:
    img_t = cv2.circle(img_t, (int(i[0]),int(i[1])), radius=5, color=(255, 0, 0), thickness=-1)
cv2.imwrite('plot.png',img_t)

True

In [ ]:
!cp /content/model.pth /content/drive/MyDrive/DL

224*(i_old - 0)/width = i_new/224

In [ ]:
model = torch.load('/content/drive/MyDrive/DL/model.pth')
model.eval()

CustomFacePointsModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=25088, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Lin

In [ ]:
img=Image.open('/content/bradpitt.png')
img = data_transform(img).unsqueeze(0)
x=model(img).detach().numpy().reshape(68,2)

#image=cv2.imread('face-morphing-multiple-images//aligned_images//bradpitt.png')
image = img.squeeze(0).detach().numpy()
image=np.transpose(image,(1,2,0))
image=(image*255).astype(int)
print(x)
for i in x:
    image = cv2.circle(image, (int(i[0]),int(i[1])), radius=0, color=(0, 0, 255), thickness=-1)

cv2.imwrite('plot.png',image)

[[[0.56078434 0.45098042 0.254902  ]
  [0.45882356 0.5921569  0.3803922 ]
  [0.4431373  0.654902   0.45882356]
  ...
  [0.22352946 0.6        0.34901965]
  [0.2313726  0.60784316 0.34901965]
  [0.24705887 0.6156863  0.35686278]]

 [[0.56078434 0.41960788 0.22352946]
  [0.5058824  0.6156863  0.39607847]
  [0.47450984 0.67058825 0.45882356]
  ...
  [0.22352946 0.62352943 0.3803922 ]
  [0.23921573 0.60784316 0.36470592]
  [0.26274514 0.60784316 0.36470592]]

 [[0.58431375 0.39607847 0.19215691]
  [0.5686275  0.62352943 0.39607847]
  [0.5294118  0.67058825 0.4431373 ]
  ...
  [0.22352946 0.654902   0.4039216 ]
  [0.23921573 0.64705884 0.4039216 ]
  [0.254902   0.6313726  0.38823533]]

 ...

 [[0.47450984 0.73333335 0.09019613]
  [0.4431373  0.7176471  0.082353  ]
  [0.39607847 0.7019608  0.07450986]
  ...
  [0.02745104 0.427451   0.18431377]
  [0.0196079  0.41960788 0.17647064]
  [0.02745104 0.427451   0.18431377]]

 [[0.47450984 0.7254902  0.09803927]
  [0.43529415 0.70980394 0.09019613]


error: ignored